In [8]:
import pandas as pd

test = pd.read_csv('test.csv')
a = pd.read_csv('autogluon_stack3.csv')
a = a[['CI_HOUR']]
#a = a.rename(columns={'CI_HOUR': 'CI_HOUR_NEW'})
test = test[['ARI_CO','ARI_PO','ATA','U_WIND', 'V_WIND','AIR_TEMPERATURE','BN']]
test = pd.concat([test,a],axis=1)

In [9]:
import pandas as pd

# 1. train_df 데이터 로드
train_df = pd.read_csv("train.csv")

# 2. train_df의 'ATA' 열을 datetime 형식으로 변환
train_df['ATA'] = pd.to_datetime(train_df['ATA'])

# 3. train_df를 기반으로 매핑 딕셔너리 생성
mapping_dict_train = {}
for _, row in train_df.iterrows():
    key = (row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN'])
    if row['CI_HOUR'] <= 1:
        mapping_dict_train[key] = row['ATA']

# 4. 후처리 함수 정의 및 적용
def compute_post_processed_target_v8(row):
    key = (row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN'])
    
    # CI_HOUR 값이 1 이하인 경우, 그대로 반환
    if row['CI_HOUR'] <= 1:
        return row['CI_HOUR']
    
    # 해당 샘플의 날짜 이후의 train_df의 데이터 중, 키가 일치하고 CI_HOUR 값이 1 이하인 데이터의 날짜 참조
    if key in mapping_dict_train and mapping_dict_train[key] > row['ATA']:
        time_difference = (mapping_dict_train[key] - row['ATA']).total_seconds() / 3600
        return abs(time_difference)
    
    # 일치하는 데이터가 없는 경우, 현재 값 반환
    return row['CI_HOUR']

# 함수 적용 및 결과 확인
test['Post_Processed_CI_HOUR_v8'] = test.apply(compute_post_processed_target_v8, axis=1)
print(test[['ATA', 'CI_HOUR', 'Post_Processed_CI_HOUR_v8']])

TypeError: '>' not supported between instances of 'Timestamp' and 'str'

In [6]:
test['ATA'] = pd.to_datetime(test['ATA'])
def compute_post_processed_target_v6(row):
    key = (row['ARI_PO'], row['U_WIND'], row['V_WIND'], row['AIR_TEMPERATURE'], row['BN'])
    
    # CI_HOUR 값이 1 이하인 경우, 그대로 반환
    if row['CI_HOUR'] <= 1:
        return row['CI_HOUR']
    
    # 해당 샘플의 날짜 이후의 데이터 중, 키가 일치하고 CI_HOUR 값이 1 이하인 데이터만 필터링
    matching_rows = test[
        (test['ARI_PO'] == key[0]) & 
        (test['U_WIND'] == key[1]) & 
        (test['V_WIND'] == key[2]) & 
        (test['AIR_TEMPERATURE'] == key[3]) & 
        (test['BN'] == key[4]) & 
        (test['CI_HOUR'] <= 1) &
        (test['ATA'] > row['ATA'])
    ]
    
    # 일치하는 데이터가 없는 경우, 현재 값 반환
    if matching_rows.empty:
        return row['CI_HOUR']
    
    # 가장 가까운 일치하는 데이터의 날짜 선택
    nearest_matching_date = matching_rows['ATA'].min()
    
    # 현재 행의 날짜와 가장 가까운 일치하는 데이터의 날짜의 차이 계산
    time_difference = (nearest_matching_date - row['ATA']).total_seconds() / 3600
    
    return abs(time_difference)

# 함수 적용 및 결과 확인
test['Post_Processed_CI_HOUR_v6'] = test.apply(compute_post_processed_target_v6, axis=1)
test[['ATA', 'CI_HOUR', 'Post_Processed_CI_HOUR_v6']]

KeyboardInterrupt: 

In [41]:
import numpy as np
old_new_df['New_Target'] = np.where((old_new_df['New_Target'] > 2159) | old_new_df['New_Target'].isna(), np.nan, old_new_df['New_Target'])
old_new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220491 entries, 0 to 220490
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ARI_CO           220491 non-null  object 
 1   ARI_PO           220491 non-null  object 
 2   ATA              220491 non-null  object 
 3   U_WIND           128766 non-null  float64
 4   V_WIND           128766 non-null  float64
 5   AIR_TEMPERATURE  128245 non-null  float64
 6   BN               128766 non-null  float64
 7   CI_HOUR_NEW      220491 non-null  float64
 8   New_Target       34865 non-null   float64
dtypes: float64(6), object(3)
memory usage: 15.1+ MB


In [40]:
#28191 / 1hour
#30567 / 6hours
#32327 / 12
#34032 / 24hours
#34834 / 48
#34865 / 72
#34758 / 96
#33355   / 8640

In [19]:
old_new_df['CI_HOUR_NEW'] = old_new_df['CI_HOUR_NEW'].where(old_new_df['New_Target'].isna(), old_new_df['New_Target'])
old_new_df

,ARI_CO,ARI_PO,ATA,U_WIND,V_WIND,AIR_TEMPERATURE,BN,CI_HOUR_NEW,New_Target
0,SG,GIW5,2020-06-18 11:58,0.37,1.63,27.1,1.587063,0.266667,0.266667
1,CN,WEY7,2021-05-26 22:20,-2.79,-2.33,14.2,2.663972,14.224622,NaN
2,CN,NGG6,2019-12-16 0:09,0.04,-4.91,9.3,3.255315,30.983333,30.983333
3,CA,FFM2,2015-11-16 5:30,NaN,NaN,NaN,NaN,84.764893,NaN
4,JP,QYY1,2018-10-24 1:11,NaN,NaN,NaN,NaN,0.000000,NaN
...,...,...,...,...,...,...,...,...,...
220486,BR,TMW2,2017-04-04 15:53,NaN,NaN,NaN,NaN,58.233528,NaN
220487,CA,GRQ5,2016-08-22 16:22,NaN,NaN,NaN,NaN,43.925217,NaN
220488,JP,VYJ1,2022-07-10 14:53,-2.87,1.22,27.4,2.405268,10.916667,10.916667
220489,CN,QQW1,2020-12-28 14:38,-2.65,-0.64,8.2,2.199039,0.000000,NaN


In [15]:
sub = pd.read_csv('sample_submission.csv')
sub['CI_HOUR'] = old_new_df['CI_HOUR_NEW']
sub.to_csv("SOTATEST_TR_PO_CHANGE_CO_24to8640.csv",index=False)
sub

,SAMPLE_ID,CI_HOUR
0,TEST_000000,4.018523
1,TEST_000001,14.224622
2,TEST_000002,30.983333
3,TEST_000003,84.764893
4,TEST_000004,0.000000
...,...,...
220486,TEST_220486,58.233528
220487,TEST_220487,43.925217
220488,TEST_220488,10.916667
220489,TEST_220489,0.000000


In [28]:
old_new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220491 entries, 0 to 220490
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ARI_CO           220491 non-null  object 
 1   ARI_PO           220491 non-null  object 
 2   ATA              220491 non-null  object 
 3   U_WIND           128766 non-null  float64
 4   V_WIND           128766 non-null  float64
 5   AIR_TEMPERATURE  128245 non-null  float64
 6   BN               128766 non-null  float64
 7   CI_HOUR_NEW      220491 non-null  float64
 8   New_Target       30772 non-null   float64
dtypes: float64(6), object(3)
memory usage: 15.1+ MB
